# Feature Engineering

The act of choosing, altering, and transforming raw data into features that may be utilised in supervised learning is known as feature engineering. It may be necessary to build and train better features in order for machine learning to perform well on new tasks. A "feature" is any measurable input that can be used in a predictive model, such as the colour of an object or the sound of someone's voice, as you may know. In simple terms, feature engineering is the process of transforming raw observations into desired characteristics through statistical or machine learning methods.

* **Feature Creation:** Feature creation entails the creation of new variables that will be most useful to our model. This could entail adding or eliminating features. 
* **Feature Transformation:** Essentially a function that converts features from one representation to the next. The idea is to plot and visualise data so that if something doesn't add up with the new features, we can reduce the number of features utilised, speed up training, or improve the accuracy of a model. (This was done in the EDA section,  we are super efficient!)
* **Benchmarking:** A Benchmark Model is the most user-friendly, dependable, clear, and interpretable model you can use to compare your own performance against. Running test datasets to see if your new machine learning model outperforms a recognised benchmark is a smart idea. These benchmarks are frequently used to compare the performance of various machine learning models such as neural networks and support vector machines, linear and non-linear classifiers, and techniques such as bagging and boosting.  We will be using LightGBM, Linear Regression, Quantile Regression and PCA to measure and benchmark performance. 

In [ ]:
# import the datasets

imp_train_data = pd.read_csv("imputed_train_data.csv") # this is the data that has only been imputed. 
eda_train_data = pd.read_csv("eda_train_data.csv") # this is the data that has been modified as per EDA insights. 

## Variance Inflation Factor(VIF)

In a multivariate regression model, multicollinearity occurs when two or more independent variables have a significant correlation between them. We may have trouble discriminating between individual influences on the dependent variable when some traits are highly associated. Various strategies can be used to detect multicollinearity, one of which is the Variance Inflation Factor (VIF).

We select each feature and regress it against all other features using the VIF approach. The factor is calculated as follows for each regression:

`VIF = 1/1-R^2`

As we can see from the calculation, the higher the R-squared value, the higher the VIF. As a result, a higher VIF indicates a higher association. This corresponds to the fact that a higher R-squared value indicates greater collinearity.

* There is no association between a given explanatory variable and any other explanatory variables in the model if the value is 1.
* A number between 1 and 5 shows that there is substantial correlation between a specific explanatory variable and other explanatory variables in the model, but it is usually not severe enough to warrant attention.
* A score of more than 5 indicates that a specific explanatory variable and other explanatory variables in the model have a potentially severe connection. The coefficient estimates and p-values in the regression output are most likely erroneous in this circumstance.

Read more about VIF [here.](https://www.statisticshowto.com/variance-inflation-factor/)

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# select what columns (if not all the dataset)
X = eda_train_data[["list_of", "columns_you_want", "to_check"]]
  
# create a VIF dataframe
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
  
# calculating VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                          for i in range(len(X.columns))]
  
print(vif_data)

Describe the results here.  Could we drop any features straight away?   Is there a chance to modify the features to reduce their VIF score?  

## Feature Creation

Features created from insight drawn from EDA - re-run the VIF to check if the features are actually good for the model. 

Ideas so far: 
* Combine monthly average temps into seasonal columns. 

## Feature Selection & Importance

Feature selection is a technique for reducing the number of features in a model and thereby lowering its computational complexity. Feature selection is frequently effective in overcoming the problem of overfitting. It assists us in discovering the minimal collection of information required to accurately predict the target variable. If a feature adds little or no benefit to the predictive power, why add them and increase the complexity?

Examining the model's coefficients is perhaps the simplest technique to analyse feature importances. Both linear and logistic regression, for example, are based on an equation in which each input value is assigned a coefficient (importance).

Simply said, if a given coefficient is a significant (negative or positive) value, it has an impact on the forecast. If the coefficient is zero, on the other hand, it has no bearing on the forecast.

### Prepare the Data

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

X = eda_train_data.iloc[:, 1:-1].values 
y = eda_train_data.iloc[:, -1].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42) 

ss = StandardScaler()
X_train_scaled = ss.fit_transform(X_train)
X_test_scaled = ss.transform(X_test)

### Light Gradient Boosting Machine (LightGBM)
LightGBM is a gradient boosting framework that uses tree based learning algorithms. It is designed to be distributed and efficient with the following advantages:

Faster training speed and higher efficiency.
* Lower memory usage.
* Better accuracy.
* Support of parallel, distributed, and GPU learning.
* Capable of handling large-scale data.

Benefiting from these advantages, LightGBM is being widely-used in many winning solutions of machine learning competitions. 

Read more about LightGBM [here.](https://www.analyticssteps.com/blogs/what-light-gbm-algorithm-how-use-it) 

In [ ]:
import lightgbm

gbm = lightgbm.LGBMRegressor()
gbm.fit(X_train_scaled, y_train)

In [ ]:
from sklearn.model_selection import cross_validate

cv_results = cross_validate(gbm, X_train_scaled, y_train, cv=5, return_estimator=True)

for model in cv_results['estimator']:
    print(model.coef_)

In [ ]:
lightgbm.plot_importance(gbm, figsize = (20, 60))
plt.show()

### Linear Regression

In the simplest terms, Linear Regression is a supervised Machine Learning model that identifies the best fit linear line between the independent and dependent variables, i.e. it discovers the linear relationship between the two variables.

Read more about linear regression [here.](https://www.analyticsvidhya.com/blog/2021/05/all-you-need-to-know-about-your-first-machine-learning-model-linear-regression/) 

In [ ]:
from sklearn.linear_model import LinearRegression

linreg = LinearRegression()

linreg.fit(X_train_scaled, y_train)
importances = pd.DataFrame(data={
    'Attribute': X_train.columns,
    'Importance': model.coef_[0]})

importances = importances.sort_values(by='Importance', ascending=False)

In [ ]:
cv_results = cross_validate(model, X_train_scaled, y_train, cv=5, return_estimator=True)

for model in cv_results['estimator']:
    print(model.coef_)

In [ ]:
# LR plot 

plt.bar(x=importances['Attribute'], height=importances['Importance'], color='blue')
plt.title('LINEAR: Feature importances obtained from coefficients', size=20)
plt.xticks(rotation='vertical')
plt.show()

### Quantile Regression

Unlike standard linear regression, which calculates the conditional mean of the target across multiple values of the characteristics using the least squares approach, quantile regression estimates the conditional median of the target. Quantile regression is a type of linear regression that is employed when the linear regression requirements aren't met (i.e., linearity, homoscedasticity, independence, or normality).

Read more about Quantile Regression [here. ](https://towardsdatascience.com/quantile-regression-from-linear-models-to-trees-to-deep-learning-af3738b527c3)

In [ ]:
from sklearn.linear_model import QuantileRegressor

quantreg = QuantileRegressor()

quantreg.fit(X_train_scaled, y_train)
importances = pd.DataFrame(data={
    'Attribute': X_train.columns,
    'Importance': model.coef_[0]})

importances = importances.sort_values(by='Importance', ascending=False)

In [ ]:
cv_results = cross_validate(quantreg, X_train_scaled, y_train, cv=5, return_estimator=True)

for model in cv_results['estimator']:
    print(model.coef_

In [ ]:
plt.bar(x=importances['Attribute'], height=importances['Importance'], color='blue')
plt.title('QUANTILE: Feature importances obtained from coefficients', size=20)
plt.xticks(rotation='vertical')
plt.show()

### Principle Component Analysis

Principal Component Analysis (PCA) is an excellent technique for reducing dimensionality and determining feature importance.

PCA, unlike the previous two algorithms, does not display you the most essential features right away. Instead, N primary components will be returned, where N is the number of original features.

Read more about PCA [here.](https://builtin.com/data-science/step-step-explanation-principal-component-analysis) 

In [ ]:
from sklearn.decomposition import PCA

pca = PCA().fit(X_train_scaled)

# compute loading scores for PCA
loadings = pd.DataFrame(
    data=pca.components_.T * np.sqrt(pca.explained_variance_), 
    columns=[f'PC{i}' for i in range(1, len(X_train.columns) + 1)],
    index=X_train.columns)

loadings.head()

In [ ]:
# plot the correlations between all features and 1st principle components

pc1_loadings = loadings.sort_values(by='PC1', ascending=False)[['PC1']]
pc1_loadings = pc1_loadings.reset_index()
pc1_loadings.columns = ['Attribute', 'CorrelationWithPC1']

plt.bar(x=pc1_loadings['Attribute'], height=pc1_loadings['CorrelationWithPC1'], color='purple')
plt.title('PCA loading scores (first principal component)', size=20)
plt.xticks(rotation='vertical')
plt.show()

But what exactly does this imply? It means that the ** count of features** components can explain 80-90% percent of the variance in the dataset.

Now to deal with the PCA loadings: these are simply the coefficients of the linear combination of the original variables used to create the major components. Loadings can be used to discover correlations between variables and main components.

In [ ]:
loadings = pd.DataFrame(
    data=pca.components_.T * np.sqrt(pca.explained_variance_), 
    columns=[f'PC{i}' for i in range(1, len(X_train.columns) + 1)],
    index=X_train.columns)

loadings.head()